**Here we compare our data to published sc and snRNA-seq data from two HFrEF (LAD-ligation (MI) and angiotensin II infusion (AngII)) and one HFpEF (high fat died / L-NAME (HFD)) model**

MI: **Forte et al., 2020**; PMID: 32130914 (E-MTAB-7895; CTRL: MF18001_1, MF18001_2; treat: MF18002_1, MF18002_2)

AngII: **McLellan et al., 2020**; PMID: 32795101 (E-MTAB-8810; CTRL: AP18002, AP18006; treat: AP18005, AP18009)

HFD: **Pepin et al., 2025**; PMID: 40340422 (integrated object, only data from C57Bl/6N mice were used) 

# Load libraries and Themes

In [3]:
suppressPackageStartupMessages({
  suppressWarnings({
    library(Seurat)
    library(SoupX)
    library(ggplot2)
    library(tidyverse)
    library(scDblFinder)
    library(harmony)
    library(SeuratDisk)
    library(SingleCellExperiment)
    library(dplyr)
    library(ggpubr)
    library(pals)
    library(viridis)
    library(scCustomize)
    library(DESeq2)
    library(EnhancedVolcano)
    library(Rsamtools)
    library(svglite)
    library(RCurl)
    library(AnnotationHub)
    library(ensembldb)
    library(networkD3)
    library(patchwork)
  })
})

In [ ]:
#Color Palettes

palette.3 <- c(
    "#440154", #G1
    "#21908d", #S
    "#fde725" #G2M
)

palette.comb <- c(
    "#aad902", #BC 
    "#990902", #CM
    "#05b1eb", #ECcap
    "#0303a3", #ECend
    "#003ae8", #EClym    
    "#ebe6c7", #FB
    "#ffab00", #MESO
    "#077a01", #MP
    "#422c00", #PER
    "#FFD100", #SC
    "#56B400"  #TC
)

palette.treatment <- c(
  "#AF0000",  # ALDO (deep red)
  "#7A9E4D",  # AngII (olive green)
  "#C1C1C1",  # CTRL (gray) 
  "#C89F67",  # HFD (soft tan/orange)
  "#4C858B",  # MI (muted blue-green)
  "#006AF3"   # REC (royal blue) 
)

palette.treatment_sub <- c(
  "#AF0000",  # ALDO (deep red)
  "#7A9E4D",  # AngII (olive green)
  "#C1C1C1",  # CTRL (gray) 
  "#C89F67",  # HFD (soft tan/orange)
  "#4C858B"  # MI (muted blue-green)
)

palette.treatment2 <- c(
    "#AF0000", 
    "#7A9E4D", 
    "#C89F67", 
    "#4C858B"
)

palette_feature <- c("lightgrey", 
    "#ffffe5",
    "#fff7bc",
    "#fee391",
    "#fec44f",
    "#fe9929",
    "#ec7014",
    "#cc4c02",
    "#993404",
    "#662506"
)

palette.sex <- c(
    "#C1C1C1", #male
    "#AF0000", #female
    "#006AF3" #mix
)

palette.FB <- c(
  "#7F4F24", # Earthy brown  
  "#B35C1E", # Rust orange  
  "#C9A27E", # Sandstone  
  "#8F6F47", # Olive brown  
  "#A7988A", # Ash beige  
  "#C48291", # Dusty rose  
  "#6C7A89", # Slate gray-blue (cool contrast)  
  "#5c2f07"  # # Warm medium tan / caramel brown
)

palette.CM <- c(
    "#67001f", # Deep red  
    "#d6604d", # Coral red  
    "#d9c9a0", # Light beige  
    "#f4a582", # Soft red  
    "#6e4742", # Greyish red  
    "#3F182B", # Dark purple  
    "#762a83"  # Muted violet
)

palette.EC <- c(
    "#00106D", 
    "#004C6D",   
    "#0097AA",   
    "#005EC4",   
    "#003366",   
    "#66A1C3",   
    "#5f729e"  
)

In [ ]:
umap_theme <- theme(
  axis.line=element_blank(),
  axis.text.x=element_blank(),
  axis.text.y=element_blank(),
  axis.ticks=element_blank(),
  axis.title.x=element_blank(),
  axis.title.y=element_blank(),
  panel.background=element_blank(),
  panel.border=element_blank(),
  panel.grid.major=element_blank(),
  panel.grid.minor=element_blank()
)

In [ ]:
setwd("/media/daten/dmeral/scseq_analysis/2024_LV_CTRL_ALDO_REC")

In [ ]:
set.seed(1234)

In [ ]:
# obj_harmony<- readRDS("comparative/seurat_objects/final_obj_all_studies.rds")

# SoupX (remove ambient signal)

In [ ]:
samples <- c("AngII_CTRL_1", "AngII_CTRL_2", "AngII_TREAT_1", "AngII_TREAT_2", 
             "MI_CTRL_1", "MI_CTRL_2", "MI_TREAT_1", "MI_TREAT_2")

In [ ]:
# Simple pre-process

mad_outlier <- function(sobj, metric, nmads){
  M <- sobj@meta.data[[metric]]
  median_M <- median(M, na.rm = TRUE)
  mad_M <- mad(M, na.rm = TRUE)
  outlier <- (M < (median_M - nmads * mad_M)) | (M > (median_M + nmads * mad_M))
  return(outlier)
}

    # Load in filtered cellranger outs
filter_mad_outliers <- function(sample_id){
  path <- paste0("/media/daten/dmeral/scseq_analysis/2024_LV_CTRL_ALDO_REC/comparative/CellRangerOuts/", sample_id, "/outs/filtered_feature_bc_matrix/")
  sobj <- Read10X(data.dir = path)
  sobj <- CreateSeuratObject(counts = sobj, min.cells = 0, min.features = 200)
  sobj$sample_id <- sample_id
  
  # add QC metrics
  sobj$log1p_total_counts <- log1p(sobj@meta.data$nCount_RNA)
  sobj$log1p_n_genes_by_counts <- log1p(sobj@meta.data$nFeature_RNA)
  sobj[["percent.mt"]] <- PercentageFeatureSet(sobj, pattern = "^mt-")
  
  # find outliers and subset
  bool_vector <- !mad_outlier(sobj, "log1p_total_counts", 5) & !mad_outlier(sobj, "log1p_n_genes_by_counts", 5) & !mad_outlier(sobj, "percent.mt", 5)
  sobj <- subset(sobj, cells = which(bool_vector))
  
  return(sobj)
}

In [ ]:
data_list <- sapply(samples, filter_mad_outliers)

In [ ]:
# Basic seurat normalization and clustering
get_soup_groups <- function(sobj){
  sobj <- NormalizeData(sobj, verbose = FALSE)
  sobj <- FindVariableFeatures(object = sobj, nfeatures = 2000, verbose = FALSE, selection.method = "vst")
  sobj <- ScaleData(sobj, verbose = FALSE)
  sobj <- RunPCA(sobj, npcs = 35, verbose = FALSE)
  sobj <- FindNeighbors(sobj, dims = 1:35, verbose = FALSE)
  sobj <- FindClusters(sobj, resolution = 0.5, verbose = FALSE)
  
  return(sobj@meta.data[["seurat_clusters"]])
}

In [ ]:
add_soup_groups <- function(sobj){
  sobj$soup_group <- get_soup_groups(sobj)
  return(sobj)
}

data_list <- sapply(data_list, add_soup_groups)

In [ ]:
make_soup <- function(sobj){
  sample_id <- as.character(sobj$sample_id[1])  # e.g., LA_2109
  path <- paste0("/media/daten/dmeral/scseq_analysis/2024_LV_CTRL_ALDO_REC/comparative/CellRangerOuts/", sample_id, "/outs/raw_feature_bc_matrix/")
  raw <- Read10X(data.dir = path)

  sc <- SoupChannel(raw, GetAssayData(sobj, layer = "counts"))
  sc <- setClusters(sc, sobj$soup_group)
  sc <- autoEstCont(sc)
  out <- adjustCounts(sc, roundToInt = TRUE)
  
  # Optional: keep original counts
  sobj[["original.counts"]] <- CreateAssayObject(counts = GetAssayData(sobj, layer = "counts"))
  
  # Set adjusted counts back to the RNA assay
  sobj <- SetAssayData(sobj, layer = "counts", new.data = out)
  
  return(sobj)
}

In [ ]:
suppressWarnings({
    data_list <- sapply(data_list, make_soup)
})

In [ ]:
# Check if  correct number of reads were removed by SoupX
# Create a data frame to store results
results <- data.frame(Sample = samples, Counts_Before = NA, Counts_After = NA, Fraction_Left = NA)

# Iterate through each sample
for (i in seq_along(samples)) {
  sample_name <- samples[i]
  data_obj <- data_list[[i]] 

  # Check if the assay exists
  if ("original.counts" %in% names(data_obj) && "RNA" %in% names(data_obj)) {
    counts_before <- sum(GetAssayData(data_obj, assay = "original.counts", layer = "counts"))
    counts_after <- sum(GetAssayData(data_obj, assay = "RNA", layer = "counts"))

    # Store results in the data frame
    results[i, "Counts_Before"] <- counts_before
    results[i, "Counts_After"] <- counts_after
    
    # Calculate percent left if counts_before is not zero
    if (!is.na(counts_before) && counts_before > 0) {
      results[i, "Fraction_Left"] <- counts_after / counts_before
    }
  } else {
    message(paste("Assay not found for sample:", sample_name))
  }
}

# Print 
print(results)

# scDblFinder (remove doublets)

In [ ]:
saveRDS(data_list, "comparative/seurat_objects/rds_data_list.rds")

In [ ]:
# data_list <- readRDS("comparative/seurat_objects/rds_data_list.rds")

In [ ]:
# Normalize/ Center and Scale Matrix
data_list_scale <- lapply(data_list, function(obj.seu) {
   obj.seu <- NormalizeData(obj.seu, verbose = FALSE)
   obj.seu <- ScaleData(obj.seu, verbose = FALSE)
   return(obj.seu)
})

In [ ]:
# Convert to SingleCellExperiment
obj_sce <- lapply(data_list_scale, function(obj.seu){
    as.SingleCellExperiment(obj.seu, assay = "RNA")
})

In [ ]:
# Run scDblFinder
obj_scDblFinder <- lapply(obj_sce, function(expression_matrix){
   scDblFinder(expression_matrix)
})

In [ ]:
# Convert to Seurat objet
obj_seu <- lapply(obj_scDblFinder, function(sce){
    as.Seurat(x = sce, 
              counts = "counts", 
              data = "logcounts")
})

In [ ]:
table_singlets <- lapply(obj_seu, function(obj){
    table(obj$scDblFinder.class)
})

print(table_singlets)

# Add meta.data, filter singlets and merge

In [ ]:
meta.data_tab <- matrix(c(rep(1, each = 8), 
                          "CTRL", "CTRL",  "AngII", "AngII", "CTRL", "CTRL", "MI", "MI", 
                          "d", "d", "d", "d","m", "m", "m", "m", 
                          rep(4, each = 4), rep(5, each = 4),
                          rep("LV", each = 8), 
                          13:20), 
                          ncol = 8, byrow = TRUE)

rownames(meta.data_tab) <- c("replicate", "treatment", "sex", "batch", "chamber", "unique")
colnames(meta.data_tab) <- samples

meta.data_tab <- as.table(meta.data_tab)

meta.data_tab

In [ ]:
# Adding metadata to Seurat objects with alignment
obj_seu_newmeta <- lapply(samples, function(nam) {
    # Retrieve the Seurat object for the current sample
    scobj <- obj_seu[[nam]]
    
    # Extract the corresponding metadata and convert it to a data frame
    meta <- meta.data_tab[, nam]
    
    # Ensure the metadata is in the right format
    meta_df <- as.data.frame(matrix(rep(meta, ncol(scobj)), nrow = ncol(scobj), byrow = TRUE))
    colnames(meta_df) <- rownames(meta.data_tab)  # Set column names to match the metadata structure
    
    # Add metadata to the Seurat object
    scobj <- AddMetaData(scobj, metadata = meta_df)
    
    return(scobj)  # Return the modified Seurat object
})

# Assign names to the new list
names(obj_seu_newmeta) <- samples

In [ ]:
# Merge in one object
obj_seu_merge <- Merge_Seurat_List(obj_seu_newmeta, add.cell.ids = c(13:20),  merge.data = TRUE, project = "comparative")

In [ ]:
saveRDS(obj_seu_merge, "comparative/seurat_objects/setContaminationFraction_dbl.rds")

# Merge with my obj and seurat obj from Pepin et al. and integration (Harmony)

In [ ]:
HFpEFstudy_obj <- readRDS("comparative/seurat_objects/NNT_Labelling_snRNA.rds") # Pepin et al. 
my_obj <- LoadH5Seurat("seurat_objects/obj_seu_merge_dbl.h5seurat") # My object

In [ ]:
# Subset HFD study object to only contain data from C57Bl/6N mice and rename meta.data
HFpEFstudy_obj <- subset(HFpEFstudy_obj, Background == "N") 
HFpEFstudy_obj$sample_id <- HFpEFstudy_obj$SampleID 

In [ ]:
# Merge my obj and pre-processed AngII/MI data
merged_obj <- merge(my_obj, y = obj_seu_merge)

In [ ]:
# Filter Singlets
obj_seu_merge_singlet <- subset(merged_obj, scDblFinder.class == "singlet")

In [ ]:
# Remove unwanted cols
cols_to_remove <- c(
  "log1p_total_counts", "log1p_n_genes_by_counts", "soup_group", 
    "nCount_original.counts", "nFeature_original.counts", "ident", 
    "scDblFinder.class", "scDblFinder.score", "scDblFinder.weighted", "scDblFinder.cxds_score"    
)

obj_seu_merge_singlet@meta.data <- obj_seu_merge_singlet@meta.data[, !(colnames(obj_seu_merge_singlet@meta.data) %in% cols_to_remove)]

In [ ]:
# Change Idents and default assay for merged object
Idents(obj_seu_merge_singlet) <- obj_seu_merge_singlet@meta.data$"unique"
DefaultAssay(object = obj_seu_merge_singlet) <- "RNA"

In [ ]:
# Change default assay for HFD study
DefaultAssay(object = HFpEFstudy_obj) <- "RNA"
HFpEFstudy_obj[["SCT"]] <- NULL

In [ ]:
# Filter to common genes (features) between studies
common_genes <- intersect(rownames(HFpEFstudy_obj), rownames(obj_seu_merge_singlet))

HFpEFstudy_obj <- subset(HFpEFstudy_obj, features = common_genes)
obj_seu_merge_singlet <- subset(obj_seu_merge_singlet, features = common_genes)

merged_final <- merge(obj_seu_merge_singlet, y = HFpEFstudy_obj)

In [ ]:
# Remove unwanted cols
cols_to_remove <- c(
  "SampleID", "Background", "Treatment", "nCount_SCT", "nFeature_SCT",
  "SCT_snn_res.0.2", "SCT_snn_res.0.4", "SCT_snn_res.0.6", "SCT_snn_res.0.8",
  "SCT_snn_res.1", "seurat_clusters", "SingleR.cluster.labels",
  "SingleR.labels", "Group", "CellType"
)

merged_final@meta.data <- merged_final@meta.data[, !(colnames(merged_final@meta.data) %in% cols_to_remove)]

In [ ]:
# Create mapping data frame for HFpEFstudy_obj
sample_metadata <- data.frame(
  sample_id = c("1", "2", "3", "7", "8", "9"),
  replicate = rep(1, each = 6),
  sex = rep("m", each = 6),
  batch = rep(7, each = 6),
  chamber = rep("LV", each = 6),
  unique = c("25", "26", "27", "28", "29", "30"),
  treatment = c(rep("CTRL", each = 3), rep("HFD", each = 3))
)

# Make sure sample_id is character in both
merged_final@meta.data$sample_id <- as.character(merged_final@meta.data$sample_id)
sample_metadata$sample_id <- as.character(sample_metadata$sample_id)

# Update values for matching sample_ids
for (col in c("replicate", "sex", "batch", "chamber", "unique", "treatment")) {
  merged_final@meta.data[merged_final@meta.data$sample_id %in% sample_metadata$sample_id, col] <-
    sample_metadata[match(
      merged_final@meta.data$sample_id[merged_final@meta.data$sample_id %in% sample_metadata$sample_id],
      sample_metadata$sample_id
    ), col]
}

In [ ]:
# Rename sample_id values from Pepin et al HFD data
merged_final@meta.data$sample_id <- recode(
  merged_final@meta.data$sample_id,
  "1" = "HFD_CTRL_1",
  "2" = "HFD_CTRL_2",
  "3" = "HFD_CTRL_3",
  "7" = "HFD_TREAT_7",
  "8" = "HFD_TREAT_8",
  "9" = "HFD_TREAT_9"
)

In [ ]:
# Filter & Integrate
obj <- subset(merged_final, subset = nFeature_RNA > 300 & nFeature_RNA < 5000 & 
                        nCount_RNA > 500 & nCount_RNA < 15000 &
                        percent.mt < 5)

obj <- NormalizeData(obj, verbose = FALSE)
obj <- FindVariableFeatures(obj, verbose = FALSE)
obj <- ScaleData(obj, verbose = FALSE)
obj <- RunPCA(obj, assay = "RNA", npcs = 30, verbose = FALSE)

In [ ]:
# Integrate "batch"
obj <- obj %>%
  RunHarmony(group.by.vars = c("batch"), theta = c(1), lambda = c(0.5), max_iter = 20, early_stop = FALSE, plot_convergence = FALSE, assay.use = "RNA", verbose = FALSE)

In [ ]:
obj_harmony <- obj %>%
  RunUMAP(reduction = "harmony", dims = 1:30, verbose = FALSE) %>%
  FindNeighbors(reduction = "harmony", dims = 1:30, verbose = FALSE) %>%
  FindClusters(resolution = 0.1)

In [ ]:
# Sanity check
options(repr.plot.width = 15, repr.plot.height = 4, repr.plot.res = 300)

DimPlot(obj_harmony, reduction = "umap", label = TRUE, label.size = 5, shuffle = TRUE, group.by = "seurat_clusters", split.by = "treatment") 

In [ ]:
# Sample-wise QC plots
options(repr.plot.width = 20, repr.plot.height = 10, repr.plot.res = 300)

VlnPlot_QC <- VlnPlot(obj_harmony, group.by = "sample_id", 
                      features = c("nCount_RNA", "nFeature_RNA", "percent.mt"),
                      pt.size = 0, raster = TRUE, ncol = 1)

plots <- lapply(1:3, function(i) {
  if (i == 3) {
    VlnPlot_QC[[i]] + theme(
      axis.text.x = element_text(size = 15),  
      axis.text.y = element_text(size = 15),  
      axis.ticks.x = element_line(),   
      axis.title.x = element_blank(),
      plot.title = element_text(size = 15)  
    )
  } else {
    VlnPlot_QC[[i]] + theme(
      axis.text.x = element_blank(), 
      axis.text.y = element_text(size = 15),  
      axis.ticks.x = element_blank(), 
      axis.title.x = element_blank(),
      plot.title = element_text(size = 15)  
    )
  }
})

combined_plot <- wrap_plots(plots, ncol = 1)

ggsave("comparative/Plots/VlnPlot_QC_by_sample_id.svg", combined_plot, units = "cm", dpi = 300, width = 20, height = 20)

suppressWarnings(print(combined_plot))

# FindAllMarkers and Rename Cluster Annotations

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 4, repr.plot.res = 300)

DimPlot(obj_harmony, reduction = "umap", label = TRUE, label.size = 5, shuffle = TRUE, raster = FALSE) 

In [ ]:
# Spli cluster 10 by thresholding umap-1
# Subset the Seurat object to include only cells from cluster 10
cluster_10_cells <- subset(obj_harmony, idents = "10")

# Get the UMAP coordinates for the subsetted cluster 10 cells
umap_coordinates <- Embeddings(cluster_10_cells, reduction = "umap")

# Assign new identities based on the UMAP_1
cluster_10_cells$ident <- ifelse(umap_coordinates[, 1] < 4, "10_2", "10_0")

# Get the barcodes (row names) of the subsetted cells (cluster 10)
subset_barcodes <- rownames(cluster_10_cells)

# Reassign the identities to the original Seurat object (obj_harmony) using a new metadata column 'split'
obj_harmony$split <- cluster_10_cells$ident

# Identify the cells in the 'split' column that have NA values (cells that were not assigned to a new identity)
na_cells <- which(is.na(obj_harmony$split))

# Transfer the 'seurat_clusters' label to the 'split' column only for cells that were not assigned a new identity
# Instead of using 'seurat_clusters', directly copy the cluster number from 'seurat_clusters'
# but ensure the cluster number aligns with the original clustering
obj_harmony$split[na_cells] <- as.character(obj_harmony$seurat_clusters[na_cells])

# Define the desired order of clusters, ensuring the correct order starts at "0"
desired_order <- c("0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10_2", "10_0", "11", "12", "13", "14", "15", "16", "17")

# Ensure that cluster "0" exists in the metadata, if not, include it explicitly (for consistency)
obj_harmony$split <- factor(obj_harmony$split, levels = c(desired_order))

# Check the new order
unique(obj_harmony$split)

In [ ]:
# Find all markers
all.markers <- FindAllMarkers(obj_harmony, only.pos = TRUE, min.pct = 0.75, logfc.threshold = 0.58)
write.csv(all.markers, "comparative/DEGs/all.markers_wilcox_merged.csv")

In [ ]:
cluster_annotations <- list(
    "0" = "CM_0",
    "1" = "FB_0",
    "2" = "EC-cap_1",
    "3" = "MΦ_0",
    "4" = "PER_0",
    "5" = "EC-end",  
    "6" = "TC", 
    "7" = "BC", 
    "8" = "EC-lym",  
    "9" = "CM_1",
    "10_0" = "EC-cap_2",
    "10_2" = "CM_2",
    "11" = "MΦ_1",
    "12" = "FB_1",
    "13" = "MΦ_2",
    "14" = "SC",
    "15" = "CM_3",
    "16" = "MESO",
    "17" = "PER_1"
)

In [ ]:
cluster_annotations <- unlist(cluster_annotations, use.names = FALSE)

Idents(obj_harmony) <- obj_harmony@meta.data$"split"
obj_harmony@meta.data$Ident_numerical <- Idents(obj_harmony)

names(cluster_annotations) <- levels(obj_harmony)
obj_harmony <- RenameIdents(obj_harmony, cluster_annotations)
obj_harmony@meta.data$cell_type <- Idents(obj_harmony)
Idents(obj_harmony) <- obj_harmony@meta.data$"Ident_numerical"

In [ ]:
cluster_annotations_comb <- list(
    "0" = "CM",
    "1" = "FB",
    "2" = "EC-cap",
    "3" = "MΦ",
    "4" = "PER",
    "5" = "EC-end",  
    "6" = "TC", 
    "7" = "BC", 
    "8" = "EC-lym",  
    "9" = "CM",
    "10_0" = "EC-cap",
    "10_2" = "CM",
    "11" = "MΦ",
    "12" = "FB",
    "13" = "MΦ",
    "14" = "SC",
    "15" = "CM",
    "16" = "MESO",
    "17" = "PER"
)

In [ ]:
names(cluster_annotations_comb) <- levels(obj_harmony)
obj_harmony <- RenameIdents(obj_harmony, cluster_annotations_comb)

obj_harmony$"cell_type_comb" <- Idents(obj_harmony)

In [ ]:
# Reorder cell types (levels) 
new_levels <- c(
    "BC",      
    "CM",    
    "EC-cap",
    "EC-end", 
    "EC-lym",
    "FB", 
    "MESO",
    "MΦ",  
    "PER",   
    "SC",     
    "TC"   
)

# Reorder the levels in the cell_type_CMcomb column
obj_harmony@meta.data$cell_type_comb <- factor(
    obj_harmony@meta.data$cell_type_comb, 
    levels = new_levels
)

# Verify the new order of levels
levels(obj_harmony@meta.data$cell_type_comb)

In [ ]:
# Seperate CTRL in treatment specific CTRLs
obj_harmony$unique <- factor(obj_harmony$unique, levels = sort(as.numeric(levels(factor(obj_harmony$unique)))))

annotations <- list(
    "1" = "ALDO_CTRL",
    "2" = "ALDO_CTRL",
    "3" = "ALDO",
    "4" = "ALDO",
    "5" = "ALDO",
    "6" = "ALDO",  
    "7" = "ALDO_CTRL", 
    "8" = "ALDO_CTRL", 
    "9" = "REC",  
    "10" = "REC",
    "11" = "REC",
    "12" = "REC",
    "13" = "AngII_CTRL",
    "14" = "AngII_CTRL",
    "15" = "AngII",
    "16" = "AngII",
    "17" = "MI_CTRL",
    "18" = "MI_CTRL",
    "19" = "MI",
    "20" = "MI",
    "25" = "HFD_CTRL",
    "26" = "HFD_CTRL",
    "27" = "HFD_CTRL",
    "28" = "HFD",
    "29" = "HFD",
    "30" = "HFD"
)
annotations <- unlist(annotations, use.names = FALSE)
Idents(obj_harmony) <- obj_harmony@meta.data$"unique"
names(annotations) <- levels(obj_harmony)
obj_harmony <- RenameIdents(obj_harmony, annotations)
obj_harmony@meta.data$treatment_sep <- Idents(obj_harmony)

In [ ]:
# Save final object 
saveRDS(obj_harmony, "comparative/seurat_objects/final_obj_all_studies.rds")

# Cellnumbers and UMAP plots 

In [ ]:
# Cellnumbers after rename
number <- table(obj_harmony@meta.data$sample_id, 
                          obj_harmony@meta.data$cell_type_comb)
write.csv(number, file = "comparative/cellnumbers/number_perCluster_sample_id.csv")

number <- table(obj_harmony@meta.data$treatment, 
                          obj_harmony@meta.data$cell_type_comb)
write.csv(number, file = "comparative/cellnumbers/number_perCluster_teatment.csv")
number

In [ ]:
# Plot UMAPs 
options(repr.plot.width = 3, repr.plot.height = 2, repr.plot.res = 300)

UMAP_rename_cell_type <- DimPlot(obj_harmony, reduction = "umap", group.by = "cell_type_comb", label = TRUE, shuffle = TRUE, cols = palette.comb, raster = FALSE) +
  umap_theme + ggtitle("UMAP cell types") + theme(text = element_text(size = 15)) & NoLegend()

UMAP_rename_treatment <- DimPlot(obj_harmony, reduction = "umap", label = FALSE, group.by = "treatment", shuffle = TRUE, cols = palette.treatment, raster = FALSE) +
  umap_theme + ggtitle("UMAP treatment") + theme(text = element_text(size = 15)) & NoLegend()

ggsave("comparative/Plots/UMAP_rename_cell_type.svg", UMAP_rename_cell_type, units = "cm", dpi = 300, width = 30, height = 20)
ggsave("comparative/Plots/UMAP_rename_treatment.svg", UMAP_rename_treatment, units = "cm", dpi = 300, width = 30, height = 20)

suppressWarnings(print(UMAP_rename_cell_type))
UMAP_rename_treatment

In [ ]:
# Plot without REC
obj_harmony_sub <- subset(x = obj_harmony, subset = treatment %in% c("REC"), invert = TRUE)

In [ ]:
UMAP_rename_treatment <- DimPlot(obj_harmony_sub, reduction = "umap", label = FALSE, group.by = "treatment", shuffle = TRUE, cols = palette.treatment_sub, raster = FALSE) +
  umap_theme + ggtitle("UMAP treatment, w/o REC") + theme(text = element_text(size = 15)) & NoLegend()

ggsave("comparative/Plots/UMAP_rename_treatment_woREC.svg", UMAP_rename_treatment, units = "cm", dpi = 300, width = 30, height = 20)

UMAP_rename_treatment

In [ ]:
# Save final object w/o REC
saveRDS(obj_harmony_sub, "comparative/seurat_objects/final_obj_all_studies_noREC.rds")

# Cell-cycle score

In [ ]:
# Download cell cycle genes for organism at https://github.com/hbc/tinyatlas/tree/master/cell_cycle. Read it in with:
cc_file <- getURL("https://raw.githubusercontent.com/hbc/tinyatlas/master/cell_cycle/Mus_musculus.csv") 
cell_cycle_genes <- read.csv(text = cc_file)

In [ ]:
# Connect to AnnotationHub
ah <- AnnotationHub()

# Access the Ensembl database for organism
ahDb <- query(ah, 
              pattern = c("Mus musculus", "EnsDb"), 
              ignore.case = TRUE)

# Acquire the latest annotation files
id <- ahDb %>%
        mcols() %>%
        rownames() %>%
        tail(n = 1)

# Download the appropriate Ensembldb database
edb <- ah[[id]]

# Extract gene-level information from database
annotations <- genes(edb, 
                     return.type = "data.frame")

# Select annotations of interest
annotations <- annotations %>%
        dplyr::select(gene_id, gene_name, seq_name, gene_biotype, description)

In [ ]:
# Get gene names for Ensembl IDs for each gene
cell_cycle_markers <- dplyr::left_join(cell_cycle_genes, annotations, by = c("geneID" = "gene_id"))

# Acquire the S phase genes
s_genes <- cell_cycle_markers %>%
        dplyr::filter(phase == "S") %>%
        pull("gene_name")
        
# Acquire the G2M phase genes        
g2m_genes <- cell_cycle_markers %>%
        dplyr::filter(phase == "G2/M") %>%
        pull("gene_name")

# Perform cell cycle scoring
obj_harmony_sub <- CellCycleScoring(obj_harmony_sub,
                        g2m.features = g2m_genes,
                        s.features = s_genes)

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 4, repr.plot.res = 300)

UMAP_cycle <- DimPlot(obj_harmony_sub, reduction = "umap", label = FALSE, group.by = "Phase", shuffle = TRUE, raster = FALSE, cols = palette.3) +
  umap_theme + ggtitle("UMAP cellcycle") + theme(text = element_text(size = 15)) 

ggsave("comparative/Plots/UMAP_cellcycle.svg", UMAP_cycle, units = "cm", dpi = 300, width = 30, height = 20)

UMAP_cycle

# Fibroblast subcluster analysis

In [ ]:
celltype <- "FB"

In [ ]:
# Subset to FB cluster
obj_sub <- subset(x = obj_harmony, subset = cell_type_comb %in% c(celltype))

In [ ]:
# Add covariate for experimental set-up to integrate along
obj_sub$batch_2 <- ifelse(test = obj_sub$batch %in% c(1:3, 7), yes = "1", no = "2")

In [ ]:
obj_sub <- NormalizeData(obj_sub, verbose = FALSE)
obj_sub <- FindVariableFeatures(obj_sub, verbose = FALSE)
obj_sub <- ScaleData(obj_sub, verbose = FALSE)
obj_sub_pca <- RunPCA(obj_sub, assay = "RNA", npcs = 35, verbose = FALSE)

In [ ]:
# Integrate "batch" and "batch_2", and "sex"
obj_sub_pca$batch <- as.factor(obj_sub_pca$batch)
obj_sub_pca$batch_2 <- as.factor(obj_sub_pca$batch_2)

obj.subcluster <- obj_sub_pca %>% 
  RunHarmony(group.by.vars = c("batch", "batch_2", "sex"), theta = c(2, 1, 2), lambda = c(1, 0.5, 1), max_iter = 20, early_stop = FALSE, plot_convergence = FALSE, assay.use = "RNA", verbose = FALSE)

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 2.5, repr.plot.res = 300) 

dim1 <- DimPlot(obj_sub, reduction = "pca", group.by = "batch", dims = c(1, 2), cols = palette.treatment) + ggtitle("PC batch before harmony") # Before
dim2 <- DimPlot(obj.subcluster, reduction = "harmony", group.by = "batch", dims = c(1, 2), cols = palette.treatment) + ggtitle("PC batch after harmony") # After

dim1
dim2

ggsave(paste0("comparative/subcluster/", celltype, "/Plots/PC_batch_before_harmony.svg"), plot = dim1, units = "cm", dpi = 300, width = 15, height = 10)
ggsave(paste0("comparative/subcluster/", celltype, "/Plots/PC_batch_after_harmony.svg"), plot = dim2, units = "cm", dpi = 300, width = 15, height = 10)

In [ ]:
# Run UMAP
obj.subcluster <- obj.subcluster %>%
  RunUMAP(dims = 1:35,  spread = 2, min.dist = 0.3, reduction = "harmony", verbose = FALSE) %>%
  FindNeighbors(dims = 1:35, reduction = "harmony", verbose = FALSE) %>%
  FindClusters(resolution = 0.15)

In [ ]:
# PLot UMAPs
options(repr.plot.width = 10, repr.plot.height = 4, repr.plot.res = 300) 

seurat_clusters <- DimPlot(obj.subcluster, pt.size = 1, , group.by = "seurat_clusters", label = TRUE, shuffle = F, label.size = 10)
treatment_leg <- DimPlot(obj.subcluster, pt.size = 1, group.by = "treatment", shuffle = TRUE, cols = palette.treatment) + umap_theme 
treatment <- DimPlot(obj.subcluster, pt.size = 1, group.by = "treatment", shuffle = TRUE, cols = palette.treatment) + umap_theme & NoLegend()
sex <- DimPlot(obj.subcluster, pt.size = 1,  group.by = "sex", shuffle = TRUE, label.size = 10, cols = palette.sex) + umap_theme
batch <- DimPlot(obj.subcluster, pt.size = 1,  group.by = "batch", shuffle = TRUE, label.size = 10, cols = palette.treatment) + umap_theme

seurat_clusters|treatment_leg|sex|batch

In [ ]:
# Rename Idents and add cell_type_sub
cluster_annotations <- c("FB_I", "FB_II", "FB_III", "FB_IV", "FB_V", "FB_VI", "FB_VII", "FB_VIII")
names(cluster_annotations) <- levels(obj.subcluster)
obj.subcluster <- RenameIdents(obj.subcluster, cluster_annotations)
obj.subcluster$cell_type_sub <- Idents(obj.subcluster)

In [ ]:
cell_type_sub <- DimPlot(obj.subcluster, group.by = "cell_type_sub", split.by = "treatment", pt.size = 1, label = TRUE, shuffle = F, label.size = 10, cols = palette.FB) + umap_theme & NoLegend()
cell_type_sub

In [ ]:
# Plot and save plots
cell_type_sub <- DimPlot(obj.subcluster, group.by = "cell_type_sub", pt.size = 1, label = TRUE, shuffle = F, label.size = 10, cols = palette.FB) + umap_theme & NoLegend()
treatment_split <- DimPlot(obj.subcluster, pt.size = 1, group.by = "treatment", split.by = "treatment", cols = palette.treatment) + umap_theme & NoLegend()

ggsave(paste0("comparative/subcluster/", celltype, "/Plots/", celltype, "_cell_type_sub.svg"), plot = cell_type_sub, units = "cm", dpi = 300, width = 18, height = 15)
ggsave(paste0("comparative/subcluster/", celltype, "/Plots/", celltype, "_seurat_clusters.svg"), plot = seurat_clusters, units = "cm", dpi = 300, width = 18, height = 15)
ggsave(paste0("comparative/subcluster/", celltype, "/Plots/", celltype, "_treatment.svg"), plot = treatment, units = "cm", dpi = 300, width = 18, height = 15)
ggsave(paste0("comparative/subcluster/", celltype, "/Plots/", celltype, "_treatment_leg.svg"), plot = treatment_leg, units = "cm", dpi = 300, width = 18, height = 15)
ggsave(paste0("comparative/subcluster/", celltype, "/Plots/", celltype, "_treatment_split.svg"), plot = treatment_split, units = "cm", dpi = 300, width = 18*6, height = 15)
ggsave(paste0("comparative/subcluster/", celltype, "/Plots/", celltype, "_sex.svg"), plot = sex, units = "cm", dpi = 300, width = 18, height = 15)
ggsave(paste0("comparative/subcluster/", celltype, "/Plots/", celltype, "_batch.svg"), plot = batch, units = "cm", dpi = 300, width = 18, height = 15)

In [ ]:
# Perform cell cycle scoring
obj.subcluster <- CellCycleScoring(obj.subcluster,
                       g2m.features = g2m_genes,
                       s.features = s_genes)

UMAP_cycle <- DimPlot(obj.subcluster, reduction = "umap", label = FALSE, group.by = "Phase", shuffle = TRUE, raster = FALSE, cols = palette.3) +
  umap_theme + ggtitle("UMAP cellcycle") + theme(text = element_text(size = 15)) 

ggsave(paste0("comparative/subcluster/", celltype, "/Plots/", celltype, "_cellcycle.svg"), UMAP_cycle, units = "cm", dpi = 300, width = 18, height = 15)

In [ ]:
# Cellnumbers
tab <- table(obj.subcluster$treatment_sep, obj.subcluster$cell_type_sub)
tab
write.csv(tab, file = paste0("comparative/subcluster/", celltype, "/cellnumber_percluster_", celltype, ".csv"))

In [ ]:
# Find all markers
all.markers <- FindAllMarkers(obj.subcluster, only.pos = FALSE)
write.csv(all.markers, paste0("comparative/subcluster/", celltype, "/DEGs/all.markers_wilcox_", celltype ,".csv"))

In [ ]:
saveRDS(obj.subcluster, paste0("comparative/subcluster/", celltype, "/Subcluster_", celltype, ".rds"))

## Plot Gene Expression Across Studies 
switch to Kernel user_R for optimized plotting options

In [1]:
suppressPackageStartupMessages({
  suppressWarnings({
    library(Seurat)
    library(SoupX)
    library(ggplot2)
    library(tidyverse)
    library(scDblFinder)
    library(harmony)
    library(SeuratDisk)
    library(patchwork)
    library(SingleCellExperiment)
    library(dplyr)
    library(ggpubr)
    library(pals)
    library(viridis)
    library(scCustomize)
    library(DESeq2)
    library(EnhancedVolcano)
    library(Rsamtools)
    library(svglite)
    library(RCurl)
    library(sva)
    library(WGCNA)
    library(hdWGCNA)
    library(cetcolor)
  })
})

In [ ]:
celltype <- "FB"
setwd("/media/daten/dmeral/scseq_analysis/2024_LV_CTRL_ALDO_REC")

In [ ]:
# Full Object
obj.harmony <- readRDS("comparative/seurat_objects/final_obj_all_studies.rds")
obj.subcluster <- readRDS(paste0("comparative/subcluster/", celltype, "/Subcluster_", celltype, ".rds"))

In [ ]:
# Density plot UMAP
options(repr.plot.width = 4, repr.plot.height = 3, repr.plot.res = 300)

Idents(obj.harmony) <- "cell_type_comb"

# generate UMAP plot
pl1 <- UMAPPlot(obj.harmony, cols = palette.comb, 
                combine = FALSE # returns full ggplot object
                )

# custom color scale
scale.col <- cet_pal(16, name = "fire")

# make plot
umap_density_plot  <- pl1[[1]] & 
  stat_density_2d(aes_string(x = "umap_1", y = "umap_2", fill = "after_stat(level)"), 
                  linewidth = 0.2, geom = "density_2d_filled", 
                  colour = "ivory", alpha = 0.4, n = 300, h = c(1.3, 1.3)) & 
  scale_fill_gradientn(colours = scale.col)

umap_density_plot
ggsave(paste0("comparative/Plots/UMAP_density_plot.svg"), plot = umap_density_plot, width = 8, height = 4, dpi = 300)

In [ ]:
unique(obj.subcluster@meta.data$treatment)

In [ ]:
# Remove REC
obj.subcluster_sub <- subset(x = obj.subcluster, subset = treatment %in% c("REC"), invert = TRUE)

In [ ]:
# Density plot UMAP
options(repr.plot.width = 10, repr.plot.height = 6, repr.plot.res = 300)

Idents(obj.subcluster_sub) <- "cell_type_sub"

# generate UMAP plot
pl1 <- UMAPPlot(obj.subcluster_sub, cols = palette.FB, 
                combine = FALSE # returns full ggplot object
                )

# custom color scale
scale.col <- cet_pal(16, name = "fire")

# make plot
umap_density_plot  <- pl1[[1]] & 
  stat_density_2d(aes_string(x = "umap_1", y = "umap_2", fill = "after_stat(level)"), 
                  linewidth = 0.2, geom = "density_2d_filled", 
                  colour = "ivory", alpha = 0.4, n = 300, h = c(1.3, 1.3)) & 
  scale_fill_gradientn(colours = scale.col)

umap_density_plot
ggsave(paste0("comparative/subcluster/", celltype, "/Plots/UMAP_density_plot_noREC.svg"), plot = umap_density_plot, width = 8, height = 4, dpi = 300)

In [ ]:
# Reorder the levels of the 'treatment' factor to your desired order
obj.subcluster_sub$treatment_sep <- factor(obj.subcluster_sub$treatment_sep, 
                                       levels = c("ALDO_CTRL", "ALDO", "HFD_CTRL", "HFD", "AngII_CTRL", "AngII", "MI_CTRL", "MI"))

In [ ]:
obj.subcluster_sub_ALDO <- subset(x = obj.subcluster_sub, subset = treatment_sep %in% c("ALDO_CTRL", "ALDO"))
obj.subcluster_sub_AngII <- subset(x = obj.subcluster_sub, subset = treatment_sep %in% c("AngII_CTRL", "AngII"))
obj.subcluster_sub_HFD <- subset(x = obj.subcluster_sub, subset = treatment_sep %in% c("HFD_CTRL", "HFD"))
obj.subcluster_sub_MI <- subset(x = obj.subcluster_sub, subset = treatment_sep %in% c("MI_CTRL", "MI"))

In [ ]:
# Define genes of interest as a list (required by AddModuleScore)
# MR target gene signature by avg expression
key_genes <- c("Per1", "Per3", "Dbp", "Zbtb16", "Hlf")
gene_list <- list(key_genes) 

# Add module score to the object
obj.subcluster_sub <- AddModuleScore(
  object = obj.subcluster_sub,
  features = gene_list,
  name = "key_genes"
)

UMAP_plot <- FeaturePlot(
  obj.subcluster_sub,
  features = "key_genes1",
  pt.size = 0.1,
  order = TRUE,
  cols = palette_feature
) +
  umap_theme +
  theme(
    text = element_text(size = 15),
    legend.position = "right"
  )

ggsave(
  filename = paste0("comparative/subcluster/", celltype, "/Plots/GOI_UMAPs/UMAP_", paste(key_genes, collapse = "_"), "_moduleScore.svg"),
  plot = UMAP_plot,
  units = "cm", dpi = 300, width = 20, height = 13
)

UMAP_plot

In [ ]:
# Plot GOI UMAPs all cells merged
options(repr.plot.width = 10, repr.plot.height = 5, repr.plot.res = 300) 

GOI <- "avg_key_genes"

palette_feature <- c("lightgrey", 
                       "#ffffe5", "#fff7bc", "#fee391", "#fec44f",
                       "#fe9929", "#ec7014", "#cc4c02", "#993404", "#662506")

p <- FeaturePlot(obj.subcluster_sub, features = GOI, pt.size = 0.1, order = TRUE, cols = palette_feature)   # obj.subcluster_sub w/o REC

ggsave(paste0("comparative/subcluster/", celltype, "/Plots/GOI_UMAPs/not_split/UMAP_noREC_", GOI, ".svg"), plot = p, units = "cm", dpi = 300, width = 10, height = 10)

## pseudo-bulk PCA plot

In [ ]:
obj.subcluster_sub_noCTRL <- subset(x = obj.subcluster, subset = treatment_sep %in% c("ALDO", "AngII", "HFD", "MI"))

In [ ]:
# Aggregate counts by `sample_id`
obj.subcluster_sub_noCTRL$tsid <- paste0(obj.subcluster_sub_noCTRL$treatment, "_", obj.subcluster_sub_noCTRL$sample_id)

cts <- AggregateExpression(
  obj.subcluster_sub_noCTRL,
  group.by = "tsid",
  assays = "RNA",
  slot = "counts",
  return.seurat = FALSE
)

cts <- cts$RNA  # Extract the counts matrix

# Filter for protein-coding genes
protein_coding_genes <- unlist(read.csv("DEGs/nothreshold/protein_coding_gene_names_filtered.txt", header = TRUE, stringsAsFactors = FALSE))
cts_filtered <- cts[rownames(cts) %in% protein_coding_genes, ]

# Log-transform (avoid log(0) issues)
log_counts <- log1p(cts_filtered)

# 4. Remove zero-variance genes
log_counts_filtered <- log_counts[apply(log_counts, 1, var) > 0, ]

# Extract batch info for each aggregated sample
agg_samples <- colnames(log_counts_filtered)
agg_metadata <- obj.subcluster_sub_noCTRL@meta.data[!duplicated(obj.subcluster_sub_noCTRL$tsid), ]
batch_info <- agg_metadata$batch[match(agg_samples, agg_metadata$tsid)]

# PCA
pca_result <- prcomp(t(log_counts_filtered), center = TRUE, scale. = TRUE)


In [ ]:
options(repr.plot.width = 5, repr.plot.height = 4, repr.plot.res = 300)

# Create a data frame for plotting
pca_df <- data.frame(
  PC1 = pca_result$x[, 1],  # First principal component
  PC2 = pca_result$x[, 2],  # Second principal component
  tsid = rownames(pca_result$x),  
  treatment <- sub("-.*", "", rownames(pca_result$x)) # Extract treatment from tsid
)

# Plot the PCA
PCA_plot <- ggplot(pca_df, aes(x = PC1, y = PC2, color = treatment, label = tsid)) +
  geom_point(size = 4, alpha = 0.8) +
  geom_text(vjust = 2, hjust = 0.5, size = 3, color = "black") +  # Adjust vjust to place labels below
  theme_minimal() +
  labs(
    title = paste0("PCA of Pseudobulk Samples for ", celltype ," Across Studies"),
    x = paste0("PC1 (", round(summary(pca_result)$importance[2, 1] * 100, 1), "% Variance)"),
    y = paste0("PC2 (", round(summary(pca_result)$importance[2, 2] * 100, 1), "% Variance)")
  ) +
  scale_color_manual(values = palette.treatment2) +  # Correct color palette
  theme(
    legend.position = "right",
    text = element_text(size = 12),
    plot.title = element_text(size = 12),  # Decrease title size
    panel.grid.minor = element_blank(),    # Remove minor gridlines only
    panel.border = element_rect(color = "black", fill = NA, size = 0.5)  # Add black border around the plot
  )

PCA_plot
ggsave(paste0("comparative/subcluster/", celltype, "/Plots/PCA_plot_pseudobulk.svg"), plot = PCA_plot, units = "cm", dpi = 300, width = 15, height = 10)

In [2]:
sessionInfo()

R version 4.2.2 (2022-10-31)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 22.04.2 LTS

Matrix products: default
BLAS/LAPACK: /media/daten/dmeral/micromamba/envs/user_R/lib/libopenblasp-r0.3.26.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats4    stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] cetcolor_0.2.0              hdWGCNA_0.3.01             
 [3] igraph_2.1.4                WGCNA_1.72-5               
 [5] fastcluster_1.2.6           dynamicTreeCut_1.63-1      
 [7] sva_3.46.0                  BiocParallel_1.32.5        
 [9] genefilter_1.80.3           mgcv_1

# Cardiomyocyte subcluster analysis

In [ ]:
celltype <- "CM"

In [ ]:
# Subset to FB cluster

obj_sub <- subset(x = obj_harmony, subset = cell_type_comb %in% c(celltype))
obj_sub <- subset(x = obj_sub, subset = batch %in% c(1:3, 7)) # Remove AngII and MI, due to CM depletion in experiments 
obj_sub <- NormalizeData(obj_sub, verbose = FALSE)
obj_sub <- FindVariableFeatures(obj_sub, verbose = FALSE)
obj_sub <- ScaleData(obj_sub, verbose = FALSE)
obj_sub_pca <- RunPCA(obj_sub, assay = "RNA", npcs = 35, verbose = FALSE)# integrate "batch" and "batch_2"
obj_sub_pca$batch <- as.factor(obj_sub_pca$batch)

In [ ]:

obj.subcluster <- obj_sub_pca %>% 
  RunHarmony(group.by.vars = c("batch", "sex"), max_iter = 20, early_stop = FALSE, plot_convergence = FALSE, assay.use = "RNA", verbose = FALSE)

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 2.5, repr.plot.res = 300) 

dim1 <- DimPlot(obj_sub, reduction = "pca", group.by = "batch", dims = c(1, 2), cols = palette.treatment) + ggtitle("PC batch before harmony") # Before
dim2 <- DimPlot(obj.subcluster, reduction = "harmony", group.by = "batch", dims = c(1, 2), cols = palette.treatment) + ggtitle("PC batch after harmony") # After

dim1
dim2

ggsave(paste0("comparative/subcluster/", celltype, "/Plots/PC_batch_before_harmony.svg"), plot = dim1, units = "cm", dpi = 300, width = 15, height = 10)
ggsave(paste0("comparative/subcluster/", celltype, "/Plots/PC_batch_after_harmony.svg"), plot = dim2, units = "cm", dpi = 300, width = 15, height = 10)

In [ ]:
# Run UMAP
obj.subcluster <- obj.subcluster %>%
  RunUMAP(dims = 1:35,  spread = 2, min.dist = 0.3, reduction = "harmony", verbose = FALSE) %>%
  FindNeighbors(dims = 1:35, reduction = "harmony", verbose = FALSE) %>%
  FindClusters(resolution = 0.1)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 4, repr.plot.res = 300) 

seurat_clusters <- DimPlot(obj.subcluster, pt.size = 1, , group.by = "seurat_clusters", label = TRUE, shuffle = F, label.size = 10)
treatment_leg <- DimPlot(obj.subcluster, pt.size = 1, group.by = "treatment", shuffle = TRUE, cols = palette.treatment) + umap_theme 
treatment <- DimPlot(obj.subcluster, pt.size = 1, group.by = "treatment", shuffle = TRUE, cols = palette.treatment) + umap_theme & NoLegend()
sex <- DimPlot(obj.subcluster, pt.size = 1,  group.by = "sex", shuffle = TRUE, label.size = 10, cols = palette.sex) + umap_theme
batch <- DimPlot(obj.subcluster, pt.size = 1,  group.by = "batch", shuffle = TRUE, label.size = 10, cols = palette.treatment) + umap_theme

seurat_clusters|treatment_leg|sex|batch

In [ ]:
obj.subcluster_orig <- obj.subcluster # Backup

In [ ]:
# Rename Idents and add cell_type_sub
cluster_annotations <- c("CM_I", "CM_II", "CM_III", "CM_IV", "CM_V", "CM_VI", "CM_VII")
names(cluster_annotations) <- levels(obj.subcluster)
obj.subcluster <- RenameIdents(obj.subcluster, cluster_annotations)
obj.subcluster$cell_type_sub <- Idents(obj.subcluster)

In [ ]:
cell_type_sub <- DimPlot(obj.subcluster, group.by = "cell_type_sub", split.by = "treatment", pt.size = 1, label = TRUE, shuffle = F, label.size = 10, cols = palette.CM) + umap_theme & NoLegend()
cell_type_sub

In [ ]:
# Plot and save plots
cell_type_sub <- DimPlot(obj.subcluster, group.by = "cell_type_sub", pt.size = 1, label = TRUE, shuffle = F, label.size = 10, cols = palette_sc) + umap_theme & NoLegend()
treatment_split <- DimPlot(obj.subcluster, pt.size = 1, group.by = "treatment", split.by = "treatment", cols = palette.treatment) + umap_theme & NoLegend()

ggsave(paste0("comparative/subcluster/", celltype, "/Plots/", celltype, "_cell_type_sub.svg"), plot = cell_type_sub, units = "cm", dpi = 300, width = 18, height = 15)
ggsave(paste0("comparative/subcluster/", celltype, "/Plots/", celltype, "_seurat_clusters.svg"), plot = seurat_clusters, units = "cm", dpi = 300, width = 18, height = 15)
ggsave(paste0("comparative/subcluster/", celltype, "/Plots/", celltype, "_treatment.svg"), plot = treatment, units = "cm", dpi = 300, width = 18, height = 15)
ggsave(paste0("comparative/subcluster/", celltype, "/Plots/", celltype, "_treatment_leg.svg"), plot = treatment_leg, units = "cm", dpi = 300, width = 18, height = 15)
ggsave(paste0("comparative/subcluster/", celltype, "/Plots/", celltype, "_treatment_split.svg"), plot = treatment_split, units = "cm", dpi = 300, width = 18*4, height = 15)
ggsave(paste0("comparative/subcluster/", celltype, "/Plots/", celltype, "_sex.svg"), plot = sex, units = "cm", dpi = 300, width = 18, height = 15)
ggsave(paste0("comparative/subcluster/", celltype, "/Plots/", celltype, "_batch.svg"), plot = batch, units = "cm", dpi = 300, width = 18, height = 15)

In [ ]:
# Calculate Percentage of Nuclei per Cluster
# Subset by treatment_sep to ensure comparability 
Idents(obj.subcluster) <- obj.subcluster@meta.data$"treatment_sep"
obj.subcluster_sub <- subset(x = obj.subcluster, downsample = min(table(obj.subcluster$treatment_sep)[table(obj.subcluster$treatment_sep) > 0])) # downsample to min count across treatment_sep
Idents(obj.subcluster) <- obj.subcluster@meta.data$"cell_type_sub"

# Create a contingency table of clusters by treatment
cluster_treatment_counts <- table(obj.subcluster_sub$cell_type_sub, obj.subcluster_sub$treatment_sep)

# Convert to percentages by dividing each value by the column sum
cluster_treatment_percent <- sweep(cluster_treatment_counts, 2, colSums(cluster_treatment_counts), FUN = "/") * 100

# Print the result
print(cluster_treatment_percent)

write.csv(cluster_treatment_percent, file = paste0("comparative/subcluster/", celltype, "/percentage_percluster_", celltype, ".csv"))

In [ ]:
# Find all markers
all.markers <- FindAllMarkers(obj.subcluster, only.pos = FALSE)
write.csv(all.markers, paste0("comparative/subcluster/", celltype, "/DEGs/all.markers_wilcox_", celltype ,".csv"))

In [ ]:
saveRDS(obj.subcluster, paste0("comparative/subcluster/", celltype, "/Subcluster_", celltype, ".rds"))

# Endothelial cells subcluster analysis

In [ ]:
celltype <- "EC"

In [ ]:
# Subset to FB cluster

obj_sub <- subset(x = obj_harmony, subset = cell_type_comb %in% c("EC-cap", "EC-end", "EC-lym"))
obj_sub <- subset(x = obj_sub, subset = batch %in% c(1:3, 7)) # Remove AngII and MI, due to CM depletion in experiments 
obj_sub <- NormalizeData(obj_sub, verbose = FALSE)
obj_sub <- FindVariableFeatures(obj_sub, verbose = FALSE)
obj_sub <- ScaleData(obj_sub, verbose = FALSE)
obj_sub_pca <- RunPCA(obj_sub, assay = "RNA", npcs = 35, verbose = FALSE)# integrate "batch" and "batch_2"
obj_sub_pca$batch <- as.factor(obj_sub_pca$batch)

In [ ]:

obj.subcluster <- obj_sub_pca %>% 
  RunHarmony(group.by.vars = c("batch", "sex"), max_iter = 20, early_stop = FALSE, plot_convergence = FALSE, assay.use = "RNA", verbose = FALSE)

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 2.5, repr.plot.res = 300) 

dim1 <- DimPlot(obj_sub, reduction = "pca", group.by = "batch", dims = c(1, 2), cols = palette.treatment) + ggtitle("PC batch before harmony") # Before
dim2 <- DimPlot(obj.subcluster, reduction = "harmony", group.by = "batch", dims = c(1, 2), cols = palette.treatment) + ggtitle("PC batch after harmony") # After

dim1
dim2

ggsave(paste0("comparative/subcluster/", celltype, "/Plots/PC_batch_before_harmony.svg"), plot = dim1, units = "cm", dpi = 300, width = 15, height = 10)
ggsave(paste0("comparative/subcluster/", celltype, "/Plots/PC_batch_after_harmony.svg"), plot = dim2, units = "cm", dpi = 300, width = 15, height = 10)

In [ ]:
# Run UMAP
obj.subcluster <- obj.subcluster %>%
  RunUMAP(dims = 1:35,  spread = 2, min.dist = 0.3, reduction = "harmony", verbose = FALSE) %>%
  FindNeighbors(dims = 1:35, reduction = "harmony", verbose = FALSE) %>%
  FindClusters(resolution = 0.08)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 4, repr.plot.res = 300) 

seurat_clusters <- DimPlot(obj.subcluster, pt.size = 1, , group.by = "seurat_clusters", label = TRUE, shuffle = F, label.size = 10)
treatment_leg <- DimPlot(obj.subcluster, pt.size = 1, group.by = "treatment", shuffle = TRUE, cols = palette.treatment) + umap_theme 
treatment <- DimPlot(obj.subcluster, pt.size = 1, group.by = "treatment", shuffle = TRUE, cols = palette.treatment) + umap_theme & NoLegend()
sex <- DimPlot(obj.subcluster, pt.size = 1,  group.by = "sex", shuffle = TRUE, label.size = 10, cols = palette.sex) + umap_theme
batch <- DimPlot(obj.subcluster, pt.size = 1,  group.by = "batch", shuffle = TRUE, label.size = 10, cols = palette.treatment) + umap_theme

seurat_clusters|treatment_leg|sex|batch

In [ ]:
# Rename Idents and add cell_type_sub
cluster_annotations <- c("EC_I", "EC_II", "EC_III", "EC_IV", "EC_V", "EC_VI", "EC_VII")
names(cluster_annotations) <- levels(obj.subcluster)
obj.subcluster <- RenameIdents(obj.subcluster, cluster_annotations)
obj.subcluster$cell_type_sub <- Idents(obj.subcluster)

In [ ]:
cell_type_sub <- DimPlot(obj.subcluster, group.by = "cell_type_sub", split.by = "treatment", pt.size = 1, label = TRUE, shuffle = F, label.size = 10, cols = palette.EC) + umap_theme & NoLegend()
cell_type_sub

In [ ]:
# Plot and save plots
cell_type_sub <- DimPlot(obj.subcluster, group.by = "cell_type_sub", pt.size = 1, label = TRUE, shuffle = F, label.size = 10, cols = palette.EC) + umap_theme & NoLegend()
treatment_split <- DimPlot(obj.subcluster, pt.size = 1, group.by = "treatment", split.by = "treatment", cols = palette.treatment) + umap_theme & NoLegend()

ggsave(paste0("comparative/subcluster/", celltype, "/Plots/", celltype, "_cell_type_sub.svg"), plot = cell_type_sub, units = "cm", dpi = 300, width = 18, height = 15)
ggsave(paste0("comparative/subcluster/", celltype, "/Plots/", celltype, "_seurat_clusters.svg"), plot = seurat_clusters, units = "cm", dpi = 300, width = 18, height = 15)
ggsave(paste0("comparative/subcluster/", celltype, "/Plots/", celltype, "_treatment.svg"), plot = treatment, units = "cm", dpi = 300, width = 18, height = 15)
ggsave(paste0("comparative/subcluster/", celltype, "/Plots/", celltype, "_treatment_leg.svg"), plot = treatment_leg, units = "cm", dpi = 300, width = 18, height = 15)
ggsave(paste0("comparative/subcluster/", celltype, "/Plots/", celltype, "_treatment_split.svg"), plot = treatment_split, units = "cm", dpi = 300, width = 18*4, height = 15)
ggsave(paste0("comparative/subcluster/", celltype, "/Plots/", celltype, "_sex.svg"), plot = sex, units = "cm", dpi = 300, width = 18, height = 15)
ggsave(paste0("comparative/subcluster/", celltype, "/Plots/", celltype, "_batch.svg"), plot = batch, units = "cm", dpi = 300, width = 18, height = 15)

In [ ]:
# Calculate Percentage of Nuclei per Cluster
# Subset by treatment_sep to ensure comparability 
Idents(obj.subcluster) <- obj.subcluster@meta.data$"treatment_sep"
obj.subcluster_sub <- subset(x = obj.subcluster, downsample = min(table(obj.subcluster$treatment_sep)[table(obj.subcluster$treatment_sep) > 0])) # downsample to min count across treatment_sep
Idents(obj.subcluster) <- obj.subcluster@meta.data$"cell_type_sub"

# Create a contingency table of clusters by treatment
cluster_treatment_counts <- table(obj.subcluster_sub$cell_type_sub, obj.subcluster_sub$treatment_sep)

# Convert to percentages by dividing each value by the column sum
cluster_treatment_percent <- sweep(cluster_treatment_counts, 2, colSums(cluster_treatment_counts), FUN = "/") * 100

# Print the result
print(cluster_treatment_percent)

write.csv(cluster_treatment_percent, file = paste0("comparative/subcluster/", celltype, "/percentage_percluster_", celltype, ".csv"))

In [ ]:
# Find all markers
all.markers <- FindAllMarkers(obj.subcluster, only.pos = FALSE)
write.csv(all.markers, paste0("comparative/subcluster/", celltype, "/DEGs/all.markers_wilcox_", celltype ,".csv"))

In [ ]:
saveRDS(obj.subcluster, paste0("comparative/subcluster/", celltype, "/Subcluster_", celltype, ".rds"))

# Transfer Subcluster Annotations to Original obj

In [ ]:
obj_full<- readRDS("comparative/seurat_objects/final_obj_all_studies.rds")
celltype <- "CM"
CM <- readRDS(paste0("comparative/subcluster/", celltype, "/Subcluster_", celltype,".rds"))
celltype <- "FB"
FB <- readRDS(paste0("comparative/subcluster/", celltype, "/Subcluster_", celltype,".rds"))
celltype <- "EC"
EC <- readRDS(paste0("comparative/subcluster/", celltype, "/Subcluster_", celltype,".rds"))

In [ ]:
# Sanity check
options(repr.plot.width = 20, repr.plot.height = 10, repr.plot.res = 100)

DimPlot(obj_full, group.by = "cell_type_comb")|
DimPlot(CM, group.by = "cell_type_sub")
DimPlot(FB, group.by = "cell_type_sub")|
DimPlot(EC, group.by = "cell_type_sub")

In [ ]:
seurat_objects <- list(CM, FB, EC) # This can be done with several obj containing subsets of full obj

set_cell_type_sub <- function(seurat_objects) {
  for (obj in seurat_objects) {
    obj@meta.data$cell_type_sub <- Idents(obj)
  }
}

set_cell_type_sub(seurat_objects)

# Check cluster annotations in obj_full
unique(obj_full@meta.data$cell_type_comb)

In [ ]:
# Subset all cells except "celltype" in cell_type_CMcomb
object_wo_celltype <- subset(x = obj_full, subset = cell_type_comb %in% c("CM", "FB", "EC-cap", "EC-end", "EC-lym"), invert = TRUE)

# Check successful removal of celltype in object_wo_celltype
unique(object_wo_celltype@meta.data$cell_type)

In [ ]:
# Create empty cell_type_sub_num column in obj_full
obj_full$cell_type_sub <- NA

# Extract meta.data
original_metadata <- obj_full@meta.data
wo_subcluster_metadata <- object_wo_celltype@meta.data
CM_metadata <- CM@meta.data
FB_metadata <- FB@meta.data
EC_metadata <- EC@meta.data

# Reset rownames to a column in all data frames
original_metadata$barcode <- rownames(original_metadata)
wo_subcluster_metadata$barcode <- rownames(wo_subcluster_metadata)
CM_metadata$barcode <- rownames(CM_metadata)
FB_metadata$barcode <- rownames(FB_metadata)
EC_metadata$barcode <- rownames(EC_metadata)

# Subset to barcode and cell_type_sub columns
original_metadata <- original_metadata[c("barcode", "cell_type_sub")]
wo_subcluster_metadata <- wo_subcluster_metadata[c("barcode", "cell_type_comb")]
CM_metadata <- CM_metadata[c("barcode", "cell_type_sub")]
FB_metadata <- FB_metadata[c("barcode", "cell_type_sub")]
EC_metadata <- EC_metadata[c("barcode", "cell_type_sub")]

# Merge df by barcode
merged_df <- merge(original_metadata, wo_subcluster_metadata, by = "barcode", all.x = TRUE)
merged_df <- merge(merged_df, CM_metadata, by = "barcode", all.x = TRUE)
merged_df <- merge(merged_df, FB_metadata, by = "barcode", all.x = TRUE)
merged_df <- merge(merged_df, EC_metadata, by = "barcode", all.x = TRUE)
# re-name col names

names(merged_df) <- c("barcode", "original", "wo_subcluster", "CM", "FB", "EC")

# create new merged column containing either original annotations or sub cluster annotations for each barcode
merged_df$merged_column <- coalesce(merged_df$wo_subcluster, merged_df$CM, merged_df$FB, merged_df$EC)

# subset to key cols
cell_type_sub_df <- merged_df[c("barcode", "merged_column")]

# re-order cell_type_sub_df to match order in obj_full 
cell_type_sub_df <- cell_type_sub_df[match(original_metadata$barcode, cell_type_sub_df$barcode), ]

# subset to key col, unlist and add new annotations to obj_full@meta.data
cell_type_sub_df <- cell_type_sub_df[c("merged_column")]
cell_type_sub_df <- unlist(cell_type_sub_df, use.names = FALSE)
obj_full$cell_type_sub <- cell_type_sub_df

In [ ]:
summary(is.na(merged_df$merged_column)) # this are CM and EC nuclei/cells from AngII and MI models since they were excluded from subclustering due to experimental differences

In [ ]:
DimPlot(obj_full, group.by = "cell_type_sub")

In [ ]:
saveRDS(obj_full, "comparative/seurat_objects/final_obj_all_studies_wSubAnn.rds")

In [4]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 22.04.2 LTS

Matrix products: default
BLAS/LAPACK: /media/daten/dmeral/micromamba/envs/scrna_dm/lib/libopenblasp-r0.3.27.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Etc/UTC
tzcode source: system (glibc)

attached base packages:
[1] stats4    stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] patchwork_1.3.0             networkD3_0.4              
 [3] ensembldb_2.26.0            AnnotationFilter_1.26.0    
 [5] GenomicFeatures_1.54.4      AnnotationDbi_1.64.1       
 [7] AnnotationHub_3.10.